In [ ]:
import json
import random
from faker import Faker

# Initialize the Faker library to generate random data
fake = Faker()

def generate_fake_email():
    fake = Faker()
    # Generate a fake email address
    email = fake.email()
    return email

# List of possible roles, industries, and skills
roles = ["Software Engineer", "Data Scientist", "Project Manager", "Senior Project Manager", "Business Analyst", "QA Engineer"]
industries = ["IT", "Construction", "Healthcare", "Finance", "Manufacturing", "Education"]
skills = ["Python", "Java", "SQL", "Machine Learning", "Project Management", "Leadership", "Data Analysis", "Communication", "Agile", "DevOps", "Risk Management", "Stakeholder Management"]
degrees = ["Bachelors", "Masters", "PhD"]
fields_of_study = ["Computer Science", "Business Administration", "Mechanical Engineering", "Electrical Engineering", "Data Science", "Economics"]
trainings = ["Leadership", "Public Speaking", "Python", "Data Science", "Java", "Machine Learning", "AWS"]
mentor_areas = ["Agile methodologies", "Python", "Data Science", "Cloud computing", "Public speaking", "AWS", "Azure", "Executive Communication"]
coaching_specialization = ["Executive communication", "Strategic thinking", "Starting a business", "Team Coaching", "Interview preparation", "Public speaking", "Leadership", "Marketing", "Sales"]
races = ['White', 'Black', 'Asian', 'Hispanic', 'Native American', 'Pacific Islander', 'Other']
marital_status = ['Single', 'Married', 'Divorced', 'Widowed']
supervisors = range(1, 11)
# Role transitions mapping (Career Pathways)
role_transitions = {
    "Project Manager": ["Senior Project Manager", "Program Manager", "Business Analyst"],
    "Senior Project Manager": ["Program Manager", "Group Project Manager", "Senior Business Analyst"],
    "Software Engineer": ["Data Scientist", "Senior Software Engineer", "Product Manager"],
    "Senior Software Engineer": ["Lead Engineer", "Project Manager"],
    "Senior Data Scientist": ["Lead Data Scientist", "Architech"],
    "Business Analyst": ["Senior Business Analyst", "Product Manager"]
}

# Experience level mapping based on roles
experience_levels = {
    "Software Engineer": "Entry-Level",
    "Senior Software Engineer": "Senior-Level",
    "Data Scientist": "Entry-Level",
    "Senior Data Scientist": "Senior-Level",
    "Project Manager": "Mid-Level",
    "Mid-level Project Manager": "Mid-Level",
    "Senior Project Manager": "Senior-Level",
    "Business Analyst": "Mid-Level",
    "QA Engineer": "Entry-Level",
    "Program Manager": "Senior-Level"
}

# Helper function to generate skills for a person
def generate_skills():
    num_skills = random.randint(3, 6)  # Randomly assign between 3 and 6 skills
    return random.sample(skills, num_skills)

# Helper function to generate education details
def generate_education():
    degree = random.choice(degrees)
    field_of_study = random.choice(fields_of_study)
    institution = fake.company()
    return {
        "degree": degree,
        "field_of_study": field_of_study,
        "institution": institution
    }

def generate_fake_dob():
    fake = Faker()
    # Generate a fake date of birth (DOB)
    dob = fake.date_of_birth(minimum_age=18, maximum_age=80)  # You can specify age range
    return dob

def generate_fake_address_components():
    fake = Faker()
    street = fake.street_address()
    city = fake.city()
    state = fake.state()
    postal_code = fake.zipcode()
    return street, city, state, postal_code

# Generate employee data with roles, skills, education, career transitions, and experience levels
employees = []
for i in range(1, 201):
    current_role = random.choice(roles)
    transition_roles = role_transitions.get(current_role, [])
    street, city, state, postal_code = generate_fake_address_components()
    # Ensure that if there are career transition roles, they are assigned
    career_options = random.sample(transition_roles, len(transition_roles)) if transition_roles else []
    is_mentor = random.choice([True, False])
    is_coach = random.choice([True, False])
    employee = {
        "Employee_id": i,
        "Employee_name": fake.name(),
        "current_role": current_role,
        "Supervisor_Employee_ID": random.choice(supervisors),
        "Email_id": generate_fake_email(),
        "EmployeeType": random.choice(["Full-Time", "Part-Time", "Contract"]),
        "DOB": generate_fake_dob(),
        "Street": street,
        "City": city,
        "State": state,
        "Postal_code": postal_code,
        "Race": random.choice(races),
        "Marital_status": random.choice(marital_status),
        "Employee_rating": random.choice(random(1,6)),

        "skills": generate_skills(),
        "education": generate_education(),
        "department": random.choice(industries),
        "career_options": career_options,  # List of possible career transitions
        "experience_level": experience_levels.get(current_role, "Mid-Level"),  # Assign experience level
        "learning_reimbursement_eligibility": random.choice([True, False]),
        "trainings": random.sample(trainings, k = 2),
        "promoted": random.choice([True, False]),
        "is_mentor": is_mentor,
        "mentor_area": random.sample(mentor_areas, k =2) if is_mentor else [],
        "is_coach": is_coach,
        "coaching_specialization": random.sample(coaching_specialization, k = 2) if is_coach else []

    }
    employees.append(employee)

# Convert the employee data to JSON format
employee_json = json.dumps(employees, indent=4)
# print(employee_json)
# Output the generated JSON to a file
# with open("enhanced_employees_with_experience.json", "w") as file:
#     file.write(employee_json)

# print("Employee data with experience levels generated and saved to enhanced_employees_with_experience.json")


In [ ]:
query = """// Step 1: Merge the organization node (it remains the same)
MERGE (org:Organization {name: 'Kritaka.ai'})
WITH org

// Step 2: Process employee information dynamically (using UNWIND to handle a list of 100 employees)
UNWIND $employeeList AS employeeData
// For each employee, ensure the employee node is either created or updated
MERGE (e:Employee {employee_id: employeeData.Employee_id})

// Step 2.1: Update employee properties
ON CREATE SET e.name = employeeData.Employee_name, 
              e.designation = employeeData.current_role, 
              e.experience_level = employeeData.experience_level, 
              e.learning_reimbursement_eligibility = employeeData.learning_reimbursement_eligibility, 
              e.promoted = employeeData.promoted, 
              e.is_mentor = employeeData.is_mentor, 
              e.is_coach = employeeData.is_coach
ON MATCH SET e.name = employeeData.Employee_name, 
              e.designation = employeeData.current_role, 
              e.experience_level = employeeData.experience_level, 
              e.learning_reimbursement_eligibility = employeeData.learning_reimbursement_eligibility, 
              e.promoted = employeeData.promoted, 
              e.is_mentor = employeeData.is_mentor, 
              e.is_coach = employeeData.is_coach

// Step 2.2: Connect employee to the organization
MERGE (e)-[:WORKS_IN]->(org)
WITH employeeData, e, org

// Step 3: Process skills - ensure each skill is unique and link it to the employee
UNWIND employeeData.skills AS skill
MERGE (s:Skill {name: skill})
MERGE (e)-[:HAS_SKILL]->(s)
WITH employeeData, e, org

// Step 4: Process education - ensure only one education node per employee
MERGE (edu:Education {degree: employeeData.education.degree, 
                      field_of_study: employeeData.education.field_of_study, 
                      institution: employeeData.education.institution})
MERGE (e)-[:HAS_EDUCATION]->(edu)
WITH employeeData, e, org

// Step 5: Process trainings - ensure each training is unique and link it to the employee
UNWIND employeeData.trainings AS training
MERGE (t:Training {skill_name: training})
MERGE (e)-[:IS_TRAINED_IN]->(t)
WITH employeeData, e, org

// Adding career options
UNWIND employeeData.career_options AS career_option
MERGE (co:Career_Option {career_name: career_option})
MERGE (e)-[:HAS_CAREER_OPTION]->(co)
WITH employeeData, e, org

// Step 6: Process coaching specializations - ensure each specialization is unique and link it to the employee
UNWIND employeeData.coaching_specialization AS specialization
MERGE (c:CoachingSpecialization {name: specialization})
MERGE (e)-[:HAS_COACHING_SPECIALIZATION]->(c)
WITH employeeData, e, org

// Step 7: Process department - ensure each department is unique and link it to the employee
MERGE (d:Department {department_name: employeeData.department})
MERGE (e)-[:WORKS_IN_DEPARTMENT]->(d)
WITH org

// Step 8: Process courses and training programs
UNWIND [
    {
        training_name: "Leadership",
        courses: ["Foundations of Effective Leadership", "Leading High-Performance Teams", "Strategic Leadership Skills for Managers"],
        reimbursement_policy: "Employees are eligible for full reimbursement upon successful completion, with a minimum attendance of 80%. Reimbursement will cover course fees only."
    },
    {
        training_name: "Public Speaking",
        courses: ["Mastering Public Speaking", "Effective Communication for Leaders", "Public Speaking and Presentation Skills"],
        reimbursement_policy: "Employees can claim reimbursement for up to 75% of course fees, subject to approval, and must demonstrate active participation."
    },
    {
        training_name: "Python",
        courses: ["Python for Beginners", "Advanced Python Programming", "Mastering Python for Data Science"],
        reimbursement_policy: "Full reimbursement of course fees is available for employees who pass the final assessment with a score of 70% or above."
    },
    {
        training_name: "Data Science",
        courses: ["Data Science Fundamentals", "Introduction to Data Analysis with Python", "Data Science for Business Insights"],
        reimbursement_policy: "Employees can receive a 50% reimbursement upon course completion, with proof of successful participation and grades above 60%."
    },
    {
        training_name: "Java",
        courses: ["Java Programming for Beginners", "Mastering Java Development", "Advanced Java Techniques"],
        reimbursement_policy: "Reimbursement of up to 80% is available upon completion of the course with a certification of achievement."
    },
    {
        training_name: "Machine Learning",
        courses: ["Introduction to Machine Learning", "Practical Machine Learning with Python", "Deep Learning Essentials"],
        reimbursement_policy: "Employees can claim up to 70% reimbursement after completing the course with a passing grade in the final project or exam."
    },
    {
        training_name: "AWS",
        courses: ["AWS Certified Solutions Architect - Foundations", "Mastering AWS Cloud Computing", "AWS for Developers: Building Cloud Applications"],
        reimbursement_policy: "Employees are eligible for full reimbursement upon successful completion and passing certification exams, with a 70% or higher score."
    }
] AS trainingData
MERGE (tp:TrainingProgram {training_name: trainingData.training_name, reimbursement_policy: trainingData.reimbursement_policy})
MERGE (org)-[:OFFERS_TRAINING_PROGRAM_FOR]->(tp)
WITH trainingData, tp, org
UNWIND trainingData.courses AS courseName
MERGE (course:Course {training_course: courseName})
MERGE (tp)-[:INCLUDES_COURSE]->(course)
WITH org

// Step 9: Process open positions
UNWIND [
    {
        role: "Software Engineer",
        department: "Engineering",
        location: "Remote",
        description: "Develop and maintain web applications, work with cross-functional teams to design scalable software solutions, and ensure code quality through best practices."
    },
    {
        role: "Marketing Manager",
        department: "Marketing",
        location: "New York, NY",
        description: "Lead marketing campaigns, develop brand strategies, manage content creation, and analyze data to improve customer engagement."
    },
    {
        role: "Data Scientist",
        department: "Data Science",
        location: "San Francisco, CA",
        description: "Analyze large datasets to extract actionable insights, build predictive models, and collaborate with product teams to improve decision-making processes."
    },
    {
        role: "Product Manager",
        department: "Product",
        location: "Chicago, IL",
        description: "Own the product roadmap, define product features, collaborate with engineering teams to launch products, and ensure that products meet customer needs."
    },
    {
        role: "Human Resources Specialist",
        department: "Human Resources",
        location: "Austin, TX",
        description: "Manage recruitment, onboarding, employee relations, and organizational development initiatives. Support the overall HR strategy to enhance employee engagement."
    }
] AS positionData
MERGE (pos:OpenPosition {
    role: positionData.role,
    department: positionData.department,
    location: positionData.location,
    description: positionData.description
})
MERGE (org)-[:HAS_OPEN_POSITION]->(pos)
MERGE (dept:Department {name: positionData.department})
MERGE (pos)-[:BELONGS_TO_DEPARTMENT]->(dept)
WITH org

// Step 10: Process bonus eligibility for roles
UNWIND [
    {
        role: "Software Engineer",
        bonus_eligibility: "Eligible for performance-based bonuses. Bonuses are awarded annually based on individual performance, team contributions, and company performance.",
        bonus_type: "Annual",
        criteria: "Meeting or exceeding performance targets, code quality, timely project delivery, and innovation."
    },
    {
        role: "Data Scientist",
        bonus_eligibility: "Eligible for performance-based bonuses. Bonuses are awarded annually based on project success, data-driven insights impact, and company financial performance.",
        bonus_type: "Annual",
        criteria: "Achieving key metrics related to data models and collaboration with product teams."
    },
    {
        role: "Project Manager",
        bonus_eligibility: "Eligible for project-based bonuses. Bonuses are awarded based on successful project delivery, client satisfaction, and adherence to deadlines and budgets.",
        bonus_type: "project-based",
        criteria: "Successful on-time project delivery, managing project costs, maintaining high-quality standards, and client feedback."
    },
    {
        role: "Senior Project Manager",
        bonus_eligibility: "Eligible for both annual and project-based bonuses. Senior Project Managers can earn a combination of performance and project completion bonuses.",
        bonus_type: "Annual and project-based",
        criteria: "Successfully leading multiple projects, managing teams, ensuring high-quality delivery, and aligning projects with business goals."
    },
    {
        role: "Business Analyst",
        bonus_eligibility: "Eligible for performance-based bonuses. Bonuses are awarded based on the impact of business analysis work, process improvements, and project outcomes.",
        bonus_type: "Annual",
        criteria: "Successful completion of key business analysis projects and support for strategic initiatives."
    },
    {
        role: "QA Engineer",
        bonus_eligibility: "Eligible for performance-based bonuses. Bonuses are awarded based on quality assurance metrics, testing efficiency, and contribution to product quality.",
        bonus_type: "Quarterly",
        criteria: "Meeting testing deadlines, and collaboration with development teams."
    }
] AS bonusData
MERGE (b:BonusEligibility {
    role: bonusData.role, 
    eligibility: bonusData.bonus_eligibility,
    bonus_type: bonusData.bonus_type,
    criteria: bonusData.criteria
})
MERGE (org)-[:HAS_BONUS_ELIGIBILITY_POLICY]->(b)
WITH org

// Step 11: Process promotion eligibility for roles
UNWIND [
    {
        role: "Software Engineer",
        promotion_criteria: "Demonstrate technical expertise, consistently deliver high-quality code, and contribute to team collaboration.",
        next_level: "Senior Software Engineer after 2-3 years with increased responsibility and leadership in projects."
    },
    {
        role: "Data Scientist",
        promotion_criteria: "Mastery in statistical analysis, machine learning models, and impactful data-driven insights.",
        next_level: "Senior Data Scientist after 2-3 years, with leadership in data strategy and mentoring."
    },
    {
        role: "Project Manager",
        promotion_criteria: "Successful project delivery, effective team leadership, and strong stakeholder management.",
        next_level: "Senior Project Manager after 2-4 years with more complex projects and strategic oversight."
    },
    {
        role: "Senior Project Manager",
        promotion_criteria: "Proven track record of managing large-scale projects, team leadership, and strategic vision.",
        next_level: "Program Manager after 3-5 years, overseeing multiple projects and driving organizational initiatives."
    },
    {
        role: "Business Analyst",
        promotion_criteria: "Effective requirement gathering, data analysis, and process improvement.",
        next_level: "Senior Business Analyst after 2-3 years, with leadership in driving business solutions."
    },
    {
        role: "QA Engineer",
        promotion_criteria: "Consistent identification of bugs, automation skills, and contributions to improving testing processes.",
        next_level: "Senior QA Engineer after 2-3 years, leading testing strategies and mentoring junior engineers."
    }
] AS promotionData
MERGE (p:PromotionEligibility {
    role: promotionData.role, 
    promotion_criteria: promotionData.promotion_criteria, 
    next_level: promotionData.next_level
})
MERGE (org)-[:HAS_PROMOTION_POLICY]->(p)
WITH org

// Step 12: Process engagement scores for departments
UNWIND [
    {department: "IT", engagement_score: 85},
    {department: "Construction", engagement_score: 75},
    {department: "Healthcare", engagement_score: 90},
    {department: "Finance", engagement_score: 80},
    {department: "Manufacturing", engagement_score: 70},
    {department: "Education", engagement_score: 88}
] AS engagementData
MERGE (d:Engagement {department_name: engagementData.department})
ON MATCH SET d.engagement_score = engagementData.engagement_score
ON CREATE SET d.engagement_score = engagementData.engagement_score
MERGE (org)-[:HAS_ENGAGEMENT_DETAIL_FOR_DEPARTMENT]->(d)"""



In [ ]:
from neo4j import GraphDatabase

class Neo4jImporter:
    def __init__(self, uri, user, password):
        # Connect to the Neo4j database
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Close the database connection
        self.driver.close()

    def run_query(self, query, parameters=None):
        # Run the Cypher query
        with self.driver.session() as session:
            session.run(query, parameters)

    def import_data(self, employee_list, query):
        # Prepare the Cypher query with the parameters for the employee data

        
        # Step 3: Run the Cypher query with the employee data passed as parameters
        self.run_query(query, parameters={'employeeList': employee_list})

# Initialize Neo4j connection (update with your own Neo4j credentials)
uri = "neo4j+s://69b7358a.databases.neo4j.io"  # Replace with your Neo4j URI
user = "neo4j"  # Replace with your Neo4j username
password = "eqwnrv9rYVbl6vZt2hsp8C_NVOk-s4BdcoDEBltPQKo"  # Replace with your Neo4j password

# Initialize the importer
importer = Neo4jImporter(uri, user, password)

# Import employee data
importer.import_data(employees, query)

# Close the connection after the operation
importer.close()






In [ ]:
{
    "Company_Name": "Kritaka.ai",
    "Courses": [
                {
                    "training_name": "Leadership",
                    "courses": [
                    "Foundations of Effective Leadership",
                    "Leading High-Performance Teams",
                    "Strategic Leadership Skills for Managers"
                    ],
                    "reimbursement_policy": "Employees are eligible for full reimbursement upon successful completion, with a minimum attendance of 80%. Reimbursement will cover course fees only."
                },
                {
                    "training_name": "Public Speaking",
                    "courses": [
                    "Mastering Public Speaking",
                    "Effective Communication for Leaders",
                    "Public Speaking and Presentation Skills"
                    ],
                    "reimbursement_policy": "Employees can claim reimbursement for up to 75% of course fees, subject to approval, and must demonstrate active participation."
                },
                {
                    "training_name": "Python",
                    "courses": [
                    "Python for Beginners",
                    "Advanced Python Programming",
                    "Mastering Python for Data Science"
                    ],
                    "reimbursement_policy": "Full reimbursement of course fees is available for employees who pass the final assessment with a score of 70% or above."
                },
                {
                    "training_name": "Data Science",
                    "courses": [
                    "Data Science Fundamentals",
                    "Introduction to Data Analysis with Python",
                    "Data Science for Business Insights"
                    ],
                    "reimbursement_policy": "Employees can receive a 50% reimbursement upon course completion, with proof of successful participation and grades above 60%."
                },
                {
                    "training_name": "Java",
                    "courses": [
                    "Java Programming for Beginners",
                    "Mastering Java Development",
                    "Advanced Java Techniques"
                    ],
                    "reimbursement_policy": "Reimbursement of up to 80% is available upon completion of the course with a certification of achievement."
                },
                {
                    "training_name": "Machine Learning",
                    "courses": [
                    "Introduction to Machine Learning",
                    "Practical Machine Learning with Python",
                    "Deep Learning Essentials"
                    ],
                    "reimbursement_policy": "Employees can claim up to 70% reimbursement after completing the course with a passing grade in the final project or exam."
                },
                {
                    "training_name": "AWS",
                    "courses": [
                    "AWS Certified Solutions Architect - Foundations",
                    "Mastering AWS Cloud Computing",
                    "AWS for Developers: Building Cloud Applications"
                    ],
                    "reimbursement_policy": "Employees are eligible for full reimbursement upon successful completion and passing certification exams, with a 70% or higher score."
                }
            ],
    "open_positions": [
                {
                    "role": "Software Engineer",
                    "department": "Engineering",
                    "location": "Remote",
                    "description": "Develop and maintain web applications, work with cross-functional teams to design scalable software solutions, and ensure code quality through best practices."
                },
                {
                    "role": "Marketing Manager",
                    "department": "Marketing",
                    "location": "New York, NY",
                    "description": "Lead marketing campaigns, develop brand strategies, manage content creation, and analyze data to improve customer engagement."
                },
                {
                    "role": "Data Scientist",
                    "department": "Data Science",
                    "location": "San Francisco, CA",
                    "description": "Analyze large datasets to extract actionable insights, build predictive models, and collaborate with product teams to improve decision-making processes."
                },
                {
                    "role": "Product Manager",
                    "department": "Product",
                    "location": "Chicago, IL",
                    "description": "Own the product roadmap, define product features, collaborate with engineering teams to launch products, and ensure that products meet customer needs."
                },
                {
                    "role": "Human Resources Specialist",
                    "department": "Human Resources",
                    "location": "Austin, TX",
                    "description": "Manage recruitment, onboarding, employee relations, and organizational development initiatives. Support the overall HR strategy to enhance employee engagement."
                }
        ],
    "bonus_eligibility_for_roles": [
            {
                "role": "Software Engineer",
                "bonus_eligibility": "Eligible for performance-based bonuses. Bonuses are awarded annually based on individual performance, team contributions, and company performance.",
                "bonus_type": "Annual",
                "criteria": "Meeting or exceeding performance targets, code quality, timely project delivery, and innovation."
            },
            {
                "role":"Data Scientist",
                "bonus_eligibility": "Eligible for performance-based bonuses. Bonuses are awarded annually based on project success, data-driven insights impact, and company financial performance.",
                "bonus_type": "Annual",
                "criteria": "Achieving key metrics related to data models and collaboration with product teams."
            },
            {
                "role": "Project Manager",
                "bonus_eligibility": "Eligible for project-based bonuses. Bonuses are awarded based on successful project delivery, client satisfaction, and adherence to deadlines and budgets.",
                "bonus_type": "project-based",
                "criteria": "Successful on-time project delivery, managing project costs, maintaining high-quality standards, and client feedback."
            },
            {
                "role":"Senior Project Manager",
                "bonus_eligibility": "Eligible for both annual and project-based bonuses. Senior Project Managers can earn a combination of performance and project completion bonuses.",
                "bonus_type": "Annual and project-based",
                "criteria": "Successfully leading multiple projects, managing teams, ensuring high-quality delivery, and aligning projects with business goals."
            },
            { 
                "role":"Business Analyst",
                "bonus_eligibility": "Eligible for performance-based bonuses. Bonuses are awarded based on the impact of business analysis work, process improvements, and project outcomes.",
                "bonus_type": "Annual",
                "criteria": "Successful completion of key business analysis projects and support for strategic initiatives."
            },
            {   "role":"QA Engineer",
                "bonus_eligibility": "Eligible for performance-based bonuses. Bonuses are awarded based on quality assurance metrics, testing efficiency, and contribution to product quality.",
                "bonus_type": "Quarterly",
                "criteria": "Meeting testing deadlines,  and collaboration with development teams."
            }
    ],
    
    "promotion_eligibility_for_roles": [
                {
                "role": "Software Engineer",
                "promotion_criteria": "Demonstrate technical expertise, consistently deliver high-quality code, and contribute to team collaboration.",
                "next_level": "Senior Software Engineer after 2-3 years with increased responsibility and leadership in projects."
                },
                {
                "role": "Data Scientist",
                "promotion_criteria": "Mastery in statistical analysis, machine learning models, and impactful data-driven insights.",
                "next_level": "Senior Data Scientist after 2-3 years, with leadership in data strategy and mentoring."
                },
                {
                "role": "Project Manager",
                "promotion_criteria": "Successful project delivery, effective team leadership, and strong stakeholder management.",
                "next_level": "Senior Project Manager after 2-4 years with more complex projects and strategic oversight."
                },
                {
                "role": "Senior Project Manager",
                "promotion_criteria": "Proven track record of managing large-scale projects, team leadership, and strategic vision.",
                "next_level": "Program Manager after 3-5 years, overseeing multiple projects and driving organizational initiatives."
                },
                {
                "role": "Business Analyst",
                "promotion_criteria": "Effective requirement gathering, data analysis, and process improvement.",
                "next_level": "Senior Business Analyst after 2-3 years, with leadership in driving business solutions."
                },
                {
                "role": "QA Engineer",
                "promotion_criteria": "Consistent identification of bugs, automation skills, and contributions to improving testing processes.",
                "next_level": "Senior QA Engineer after 2-3 years, leading testing strategies and mentoring junior engineers."
                }
            ],

    "Engagement_score_for_departments": [
            {
            "department": "IT",
            "engagement_score": 85
            },
            {
            "department": "Construction",
            "engagement_score": 75
            },
            {
            "department": "Healthcare",
            "engagement_score": 90
            },
            {
            "department": "Finance",
            "engagement_score": 80
            },
            {
            "department": "Manufacturing",
            "engagement_score": 70
            },
            {
            "department": "Education",
            "engagement_score": 88
            }
        ]
}

{'Company_Name': 'Kritaka.ai',
 'Courses': {'Leadership': 'Foundations of Effective Leadership, Leading High-Performance Teams, Strategic Leadership Skills for Managers',
  'Public Speaking': 'Mastering Public Speaking, Effective Communication for Leaders, Public Speaking and Presentation Skills',
  'Python': 'Python for Beginners, Advanced Python Programming, Mastering Python for Data Science',
  'Data Science': 'Data Science Fundamentals, Introduction to Data Analysis with Python, Data Science for Business Insights',
  'Java': 'Java Programming for Beginners, Mastering Java Development, Advanced Java Techniques',
  'Machine Learning': 'Introduction to Machine Learning, Practical Machine Learning with Python, Deep Learning Essentials',
  'AWS': 'AWS Certified Solutions Architect - Foundations, Mastering AWS Cloud Computing, AWS for Developers: Building Cloud Applications'},
 'open_positions': [{'role': 'Software Engineer',
   'department': 'Engineering',
   'location': 'Remote',
   'd